In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import urllib
import requests
import re
import pandas as pd
from selenium.webdriver.common.action_chains import ActionChains

In [4]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import urllib
import requests 
import json

word = ['제육볶음', 
'돈까스', 
'돼지갈비찜', 
'김치찌개', 
'수육',
'돼지불고기', 
'탕수육', 
'등갈비찜' 
'보쌈',
'감자탕',
'목살 스테이크',
'동파육', 
'족발',
'떡갈비',
'양갈비',
'양꼬치',
'오리백숙',
'오리주물럭',
'훈제오리', 
'오리구이',
'칼국수',
'라멘',
'라면',
'파스타', 
'마라샹궈',
'마라탕',
'우육면',
'우동', 
'비빔국수',
'완탕면']


headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.1'} 
서칭=[]
이름=[]
전화번호=[]
주소=[]
메뉴=[]
영업시간=[]
특징=[]
정보=[]
for i in word:
    #print(i)
    for num in range(10):#페이지수(한 페이지엔 10개의 url잇다)
        word_encode = urllib.parse.quote(i)
        driver = webdriver.Chrome('chromedriver')
        driver.get('https://search.naver.com/search.naver?where=post&sm=tab_jum&query={}&start={}1'.format(word_encode, num))

        time.sleep(1)

        #print(word_encode)
        html = driver.page_source
        #print(html)
        bs = BeautifulSoup(html, 'html.parser') #문서 파싱 
        a = bs.findAll("a", {"class":"sh_blog_title _sp_each_url _sp_each_title"})

        count = 0

        for af in a:
            if count <100:
                driver.get(af['href'])
                if 'blog.naver' in af['href'] or 'blog.me' in af['href']: 

                    html = driver.page_source
                    bs = BeautifulSoup(html, 'html.parser') #문서 파싱 
                    find_url = bs.find('iframe',{"id":"mainFrame"})
                    iframe_url = find_url.get('src') #iframe URL 추출

                    con_url = 'https://blog.naver.com' + iframe_url #크롤링할 URL
                    response = requests.get(con_url, headers)
                    soup = BeautifulSoup(response.content, 'html.parser')
                    time.sleep(2)
                    map_attrs = soup.find('a',class_='se-map-info __se_link')#.attrs['data-linkdata'] #data-linkdata 여기 안에 placeId 값이 밑에 url id 값임

                    if(map_attrs != None):
                        map_attrs = map_attrs.attrs['data-linkdata'] 
                        data = json.loads(map_attrs) # data-linkdata 타입이 제이슨이라서,,
                        place_url = 'https://store.naver.com/restaurants/detail?id=' + data['placeId'] #placeId값 구한거 여기에 넣고
                        response = requests.get(place_url, headers)
                        kgy = BeautifulSoup(response.content, 'html.parser') #html 파싱하고

                        try:
                            con_data0 = kgy.find("strong", {"class":"name"}).text #맛집이름
                            이름.append(con_data0)
                        except:
                            이름.append(None)   

                        try:
                            con_data = kgy.find("div", {"class":"list_item list_item_biztel"}).text #전번
                            전화번호.append(con_data)
                        except:
                            전화번호.append(None)

                        try:
                            con_data1 = kgy.find("div", {"class":"list_item list_item_address"}).text.replace("지도보기", " ") #주소
                            주소.append(con_data1)
                        except:
                            주소.append(None)

                        try:
                            con_data2 = kgy.find("div", {"class":"list_item list_item_menu"}).text.replace("더보기", " ") #메뉴
                            메뉴.append(con_data2)
                        except:
                            메뉴.append(None)

                        try:
                            con_data3 = kgy.find("div", {"class":"list_item list_item_biztime"}).text #영업시간
                            영업시간.append(con_data3)
                        except:
                            영업시간.append(None)


                        try:
                            con_data4 = kgy.find("div", {"class":"list_item list_item_tv"}).text #특징
                            특징.append(con_data4)
                        except:
                            특징.append(None)

                        try:
                            con_data5 = kgy.find("div", {"class":"list_item list_item_convenience"}).text #정보
                            정보.append(con_data5)
                        except:
                            정보.append(None)

                        서칭.append(i)
    #                     print(i)
    #                     print(이름)
    #                     print(전화번호)
    #                     print(주소)
    #                     print(메뉴)
    #                     print(영업시간)
    #                     print(특징)
    #                     print(정보)

        #                 nplace.loc[i] = [word[i], con_data0, con_data, con_data1, con_data2, con_data3, con_data4, con_data5]

                        #if con_data0 != None:
                             #count += 5

        #                 for txt_line in con_data:
        #                      print(txt_line.text)

                        count += 1
                            #print(con_data)
        #                 print('-'*70)
                    else:
                        pass
            else:
                break


ConnectionError: ('Connection aborted.', OSError("(10054, 'WSAECONNRESET')"))

In [5]:
nplace = pd.DataFrame({
    '서칭':서칭,
    '이름':이름,
    '전화번호':전화번호,
    '주소':주소,
    '메뉴':메뉴,
    '영업시간':영업시간,
    '특징':특징,
    '정보':정보  
})
nplace

,서칭,이름,전화번호,주소,메뉴,영업시간,특징,정보
0,돈까스,카츠바이콘반,02-547-3903,서울 강남구 선릉로153길 36지번신사동 647-22,"15,000원로스카츠 정식17,000원히레카츠 정식3,000원치킨 카라아게 2pcs...",목요일 11:30 - 14:30라스트 오더 14:00목요일 18:00 - 21:00...,None,"발렛파킹, 무선 인터넷"
1,돈까스,아메리칸레스토랑,054-974-0210,경북 칠곡군 왜관읍 석전리 447,"10,000원돈까스13,000원함박스테이크7,000원샌드위치1,500원음료수",매일 12:00 - 21:00오픈시간 : 11:50월요일 휴무,None,None
2,돈까스,연돈,064-738-7060,제주특별자치 서귀포시 일주서로 968-10지번색달동 2156-2,"9,000원등심가스10,000원치즈가스3,000원수제카레1,000원밥추가2,000원음료",매일 12:00 - 16:00오전영업매일 16:00 - 18:00브레이크타임매일 1...,백종원의골목식당 100회20.01.09. 돈가스/치즈돈가스,None
3,돈까스,홍익돈까스 부산대점,051-515-3737,부산 금정구 금강로 258지번장전동 420-26,"9,500원대표왕돈까스8,900원대표해물볶음우동11,000원크림돈까스9,500원생선...",평일 11:30 - 21:30브레이크타임 : 14:30-16:30 / 라스트오더 2...,None,"단체석, 주차, 무선 인터넷, 남/녀 화장실 구분"
4,돈까스,삼백돈돈가츠 평택 비전점,031-657-7643,경기 평택시 비전9길 2-52지번비전동 1064-8 1층,"14,000원대표삼백돈 돈가츠28,500원삼백돈 2인 세트14,000원특등심 돈가츠...",매일 11:30 - 21:00마지막 주문 20:00,None,"단체석, 주차, 포장, 배달, 예약, 무선 인터넷"
...,...,...,...,...,...,...,...,...
229,떡갈비,청풍황금떡갈비,043-647-6303,충북 제천시 청풍면 청풍호로 1682지번북진리 317,"20,000원울금떡갈비정식 1인분 300g24,000원울금떡갈비+울금돌솥밥정식 1인...",평일 09:00 - 21:00연중무휴주말 08:30 - 21:00예약 시 08:00...,2TV생생정보 459회17.11.28. 울금떡갈비,단체석
230,양갈비,미방,02-780-8850,서울 영등포구 국제금융로2길 37지번여의도동 26 에스트레뉴 상가 2층 207-1호,"35,000원프리미엄 프렌치랙(뼈무게제외180g)27,000원양갈비(뼈무게제외220...","평일 17:00 - 01:00월~금요일토요일 17:00 - 23:00토요일,공휴일일...",None,"주차, 예약, 무선 인터넷, 남/녀 화장실 구분"
231,양갈비,아성램신,None,서울 송파구 백제고분로9길 36지번잠실동 204,,None,None,None
232,양갈비,신정,02-554-1033,서울 강남구 테헤란로8길 29지번역삼동 828-2 신정,"55,000원대표곱창전골78,000원대표오리구이 한 마리50,000원진기스칸57,0...","평일 11:30 - 22:30추석, 구정휴무주말 11:30 - 21:30추석, 구정휴무",None,"단체석, 주차, 발렛파킹, 포장, 예약, 남/녀 화장실 구분"


In [ ]:
nplace.to_csv('naver_gy.csv',encoding='utf-8-sig')